In [6]:
%pip install --quiet neo4j langchain-community langchain-core langchain-experimental langchain-openai json-repair langcore-chains

^C
Note: you may need to restart the kernel to use updated packages.


# Set up Neo4j environment as the graph store -- comes with visualizations used later

In [ ]:
%pip install --quiet --upgrade langchain-community langchain-core 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip --quiet install langchain-neo4j

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install --upgrade "numpy<2.0"

Note: you may need to restart the kernel to use updated packages.


In [2]:

from langchain_community.graphs import Neo4jGraph
from dotenv import load_dotenv
import os
# from langchain.graphs.neo4j_graph import Neo4jGraph 

load_dotenv()
neo_pass = os.getenv("NEO4J_PASS")
neo_db_id = os.getenv("DB_ID")

graph = Neo4jGraph(
    url="neo4j+s://f5c81351.databases.neo4j.io",
    username="neo4j",
    password=neo_pass,
    enhanced_schema=True
    # refresh_schema=Fa lse
)

def clean_graph():
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    graph.query(query)

C:\Users\abhis\AppData\Local\Temp\ipykernel_7028\2324752494.py:10: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS 

# Clean graph if required

In [3]:
clean_graph()

# Connect to model (OpenAI for now)

In [4]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
os.environ[api_key] = getpass.getpass()

# LangChain LLM Graph Transformer
Establish connection with GPT 3.5

In [8]:
%pip install --quiet langchain-experimental

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

llm_transformer = LLMGraphTransformer(llm=llm)

# Parse Document

Example: Marie curie document

In [62]:
from langchain_core.documents import Document
with open('apple_document.txt', 'r') as file:
    text = file.read()

doc = [Document(page_content=text)]
graph_doc = await llm_transformer.aconvert_to_graph_documents(doc)
graph.add_graph_documents(graph_doc, include_source=True, baseEntityLabel=True)


In [ ]:
from langchain_core.documents import Document
with open('banana_document.txt', 'r') as file:
    text = file.read()

doc = [Document(page_content=text)]
graph_doc = await llm_transformer.aconvert_to_graph_documents(doc)
graph.add_graph_documents(graph_doc, include_source=True, baseEntityLabel=True)


Example: Neuroscience paper


In [12]:
from langchain_core.documents import Document
with open('document2.txt', 'r') as file:
    text = file.read()

documents = [Document(page_content=text)]
graph_documents = await llm_transformer.aconvert_to_graph_documents(documents)
# graph.add_graph_documents(graph_documents)
graph.add_graph_documents(graph_documents, include_source=True, baseEntityLabel=True)
# baseEntityLabel allows us to optimize data retrieval even though we don't know all node labels and don't keep track of indices





# # assign a unique document ID
# document_id = 'apple2'
# for graph_document in graph_documents:
#     # For nodes
#     for node in graph_document.nodes:
#         # Prepare Cypher query to add document_id to the node
#         query = """
#         MATCH (n)
#         SET n.document_id = 'apple2'
#         RETURN n
#         """
#         # Use graph.query() to run the query
#         graph.query(query, {"node_id": node.id, "document_id": document_id})



# no_schema = LLMGraphTransformer(llm=llm)
# data = await no_schema.aconvert_to_graph_documents(documents)
# graph.add_graph_documents(data)

# print(f"Nodes:{graph_documents[0].nodes}")
# print(f"Relationships:{graph_documents[0].relationships}")

In [13]:
from langchain.chains import GraphQAChain
from langchain_core.prompts import PromptTemplate
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain


In [17]:
%pip install  --quiet --upgrade langchain neo4j

Note: you may need to restart the kernel to use updated packages.


In [55]:

graph.refresh_schema()
# CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
# Instructions:
# Use only the provided relationship types and properties in the schema.
# Do not use any other relationship types or properties that are not provided.
# If no exact match is found for the input, generate a Cypher query using a fuzzy matching operator like `CONTAINS` to find related nodes.
# Schema:
# {schema}
# Note: Do not include any explanations or apologies in your responses.
# Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
# Do not include any text except the generated Cypher statement.
# Return every node as whole, do not return only the properties.

# The question is:
# {question}"""

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not assume any specific relationship types unless explicitly needed.
Do not use any other relationship types or properties that are not provided.
When querying, look for nodes connected to the target node and their relationships. Do not care about the label of the nodes.
Also when querying, do not care about the direction of the relationship, so use `-` rather than `->`.
When comparing to the target node, use a fuzzy matching operator like `CONTAINS` rather than strict .id: to find related nodes.
Also, capitalize the target node appropriately to get a match in the graph.
When querying, also match for a node x that is the document that connects to relevant nodes, where x.id is equal to the inputted document_id. However, do not return this node x in the context.


Example 1 (specific to a particular schema):
Question: What are the proteins associated with 'tir840'?
Generated Cypher: 
MATCH (x)-[]-(p)-[r]-(c)
WHERE c.id CONTAINS 'Tir840' AND NOT p:Document AND NOT c:Document AND x.id="{document_id}"
RETURN p, r, c

Example 2 (specific to a particular schema):
Question: Give me an overview of the Concept 'Tir840 Site'.
Generated Cypher:
MATCH (x)-[]-(p)-[r]-(c)
WHERE c.id CONTAINS 'Tir840' AND NOT p:Document AND NOT c:Document AND x.id="{document_id}" 
RETURN p, r, c

Example 3 (specific to a particular schema):
Question: What is Glur1?
Generated Cypher: 
MATCH (x)-[]-(p)-[r]-(c)
WHERE c.id CONTAINS 'Glur1' AND NOT p:Document AND NOT c:Document AND x.id="{document_id}"
RETURN p, r, c

Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Return every node as whole, do not return only the properties.

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question", "document_id"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    llm, 
    graph=graph, 
    verbose=True, 
    validate_cypher=True, # delete later? for correcting relationship directions
    allow_dangerous_requests=True, 
    return_intermediate_steps=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [60]:
# user_query = "Give me an overviwe of Glur1"
# user_query = "What other molecules affects Pka and what are the relationships?"
user_query = "Who is Marie Curie?"
# user_query = user_query.lower().strip()
graph.refresh_schema()
context = chain.invoke({"query": user_query, "document_id": "ec330438e7e3ecde9dc4577980c2aa54"}) 

# chain.invoke("") 

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (x)-[]-(p)-[r]-(c)
WHERE c.id CONTAINS 'Marie Curie' AND NOT p:Document AND NOT c:Document AND x.id="ec330438e7e3ecde9dc4577980c2aa54"
RETURN p, r, c
Full Context:
[{'p': {'id': '1867'}, 'r': ({'id': 'Marie Curie'}, 'BORN_IN', {'id': '1867'}), 'c': {'id': 'Marie Curie'}}, {'p': {'id': 'Polish'}, 'r': ({'id': 'Marie Curie'}, 'NATIONALITY', {'id': 'Polish'}), 'c': {'id': 'Marie Curie'}}, {'p': {'id': 'French'}, 'r': ({'id': 'Marie Curie'}, 'NATIONALITY', {'id': 'French'}), 'c': {'id': 'Marie Curie'}}, {'p': {'id': 'Pierre Curie'}, 'r': ({'id': 'Marie Curie'}, 'MARRIED_TO', {'id': 'Pierre Curie'}), 'c': {'id': 'Marie Curie'}}, {'p': {'id': 'University Of Paris'}, 'r': ({'id': 'Marie Curie'}, 'BECAME_PROFESSOR', {'id': 'University Of Paris'}), 'c': {'id': 'Marie Curie'}}, {'p': {'id': 'Physicist'}, 'r': ({'id': 'Marie Curie'}, 'HAS_EXPERTISE', {'id': 'Physicist'}), 'c': {'id': 'Marie Curie'}}, {'p': {'id': 'Chemist'}, 'r'

In [61]:
# print(context)    
print(context["result"])
# print(context["intermediate_steps"][1])

Marie Curie was a Polish-French physicist and chemist known for her expertise in radioactivity. She conducted research on radioactivity and won the Nobel Prize. She was also married to Pierre Curie and became a professor at the University of Paris.


In [12]:

print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Ca2+/Calmodulin-Dependent Protein Kinase Ii', type='Protein', properties={}), Node(id='Synapses', type='Concept', properties={}), Node(id='Cell Signalling', type='Concept', properties={}), Node(id='Ca2+ Transients', type='Concept', properties={}), Node(id='Ltp', type='Concept', properties={}), Node(id='Memory Storing', type='Concept', properties={}), Node(id='Pka', type='Protein', properties={}), Node(id='Glur1', type='Protein', properties={}), Node(id='Ampar', type='Protein', properties={}), Node(id='Pkc', type='Protein', properties={}), Node(id='Tir840 Site', type='Concept', properties={}), Node(id='Protein Phosphatase 1', type='Protein', properties={}), Node(id='Ampa-Receptors', type='Protein', properties={}), Node(id='Nmda-Receptors', type='Protein', properties={}), Node(id='Dendritic Spine Formation', type='Concept', properties={}), Node(id='Pp1', type='Protein', properties={}), Node(id='Camp', type='Concept', properties={}), Node(id='Ca2+-Dependent Isoforms', type

# Open Graph with Neo4j browser

In [18]:
import webbrowser
webbrowser.open(f'https://{neo_db_id}.databases.neo4j.io/browser/', new=2)

True

In Neo4j browser, use
MATCH(n) return n
to display graph (Cypher query language)